# DATABASE ORGANIZATION

In this notebook, different preprocessing techniques are tried and used for organizing the image dataset that is going to be fed to the models. Models are not built in this notebook, and the TFRecords files are not created here either. This is just some previous experimentation and the organization of the RAW data into class folders. It is also important to keep in mind that some of this code ended up not being used for the final work.

In the next cell, the neccesary libraries are imported:

* `SimpleITK` for managing `.nii` files.
* `numpy` for matrix operations. It is neccesary for `SimpleITK` to work.
* `pandas` for loading tables with basic information about the images, like their label.
* `matplotlib` for image visualization.
* `dltk.io.preprocessing` for some useful functions, like whitening.
* `skimage.filters`, to try some filters on the images.
* `os` for file interaction.

In [0]:
import SimpleITK as sitk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dltk.io import preprocessing
from skimage import filters

import os

Sections marked with the DEPRECATED word in the title were not used for the final work. Still, it required some hours, so the code was not removed. 

---

## Image load experimentation

`SimpleITK` is supposed to be a great alternative for working with `.nii` images, which is the format provided by ADNI. Let's try it. Load the path and name of a random image.

In [0]:
# path to .nii file
IMAGE = '/path/to/image/ADNI_XXXXX_XXXXX_XXXXX.nii'

Try `SimpleITK`:

In [0]:
# load in sitk format
sitk_image = sitk.ReadImage(IMAGE)
# transform into a numpy array
img = sitk.GetArrayFromImage(sitk_image)
# check the final shape
img.shape

### Image Visualization

Now, let's try visualizing the images. Using `matplotlib.pyplot` is a very simple and perfectly valid option, but selecting a certain slice. There are three dimensions that can be used for slicing. In the following cell, slice 70 of the third dimension is shown.

In [0]:
plt.imshow(img[:, :, 70], cmap='gray')
plt.show()

Let´s try appliying otsu´s thresholding, like in Ding et al. (2018). It should binarize the image. This tecnhique was not used in the final work, this was just a test.

In [0]:
otsu = filters.threshold_otsu(img)
otsu_img = img > otsu
plt.imshow(otsu_img[:, :, 70], cmap='gray')
plt.show()

Now, a very important preprocessing step for the images will be space normalization. For that, the following `resample_img` method will be used. As mentioned in the complete paper, it is taken from a sample code at [this post](https://medium.com/tensorflow/an-introduction-to-biomedical-image-analysis-with-tensorflow-and-dltk-2c25304e7c13).

In [0]:
def resample_img(itk_image, out_spacing=[2.0, 2.0, 2.0]):
    ''' This function resamples images to 2-mm isotropic voxels.
      
        Parameters:
            itk_image -- Image in simpleitk format, not a numpy array
            out_spacing -- Space representation of each voxel
            
        Returns: 
            Resulting image in simpleitk format, not a numpy array
    '''
    
    # Resample images to 2mm spacing with SimpleITK
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(itk_image)

In the following cell, the method is tried out with the previously loaded image. Also, tried cropping and whitening using `DLTK` methods. Finally, the result is printed, showing the 100th slice of the second axis.

In [0]:
res = resample_img(sitk_image)
res_img = sitk.GetArrayFromImage(sitk_image)
res_img = preprocessing.resize_image_with_crop_or_pad(res_img, img_size=(128, 192, 192), mode='symmetric')
res_img = preprocessing.whitening(res_img)
plt.imshow(res_img[:, 100, :], cmap='gray')
plt.show()

---

## Dataset organization and Drive upload (DEPRECATED)

Having bought extra storage space on Google Drive, read every image in the MRI ADNI database that has been downloaded. Then, all of them will be copied into a separate folder structure, where images from the CN class will be in one folder, the ones from the MCI class in another, and the ones from the AD class in another. Later, this folder structure will be uploaded to Google Drive for working with Google Colab.


In [0]:
CN_FOLDER = 'CN'
MCI_FOLDER = 'MCI'
AD_FOLDER = 'AD'

Create a few constants with the images paths.

In [0]:
# gdrive path, one for shell commands and the other with python format
DRIVE_SHELL_PATH = '/path/to/gdrive/'
DRIVE_PATH = '/path/to/gdrive/'

# drive path for the folder with the skull stripped images
DRIVE_SS_PATH = '/path/to/gdrive/skull_stripped/'
# file path of the description file
# it contains the information about the images
DESCRIPTION_FILE = '/path/to/Description.csv'

# dataset path, which in this case was stored in a external drive
DATASET_PATH = '/path/to/organized/dataset'
# raw directories where the images where previously organized
DATASET_FOLDERS = ['1', '2', '3', '4', '5']

Open the `.csv` file with the main information about the dataset.

In [0]:
description = pd.read_csv(DESCRIPTION_FILE)
description.head()

Finally, the following method can be used for processing and uploading the images. It extracts the image ID from the filename, checks the label (AD/CN/MCI) and copies the image to its corresponding destination folder. It also can skull-strip the image, but this option should be ignored unless the image has previously been registered. Both the code for registration and skull-stripped can be found below, in this same notebook.

In [0]:
def process_and_upload(filename, path, skull_stripping=True, random_printing=False):
    ''' Process the image name and copy the image to its
        corresponding Google Drive folder.
        
        Parameters:
            filename -- Name of the image file (.nii)
            path -- The path were the image is located
            skull_stripping -- Whether or not to practice skull stripping
                               (The skull stripping method is defined in the
                               next section)
            random_priting -- 10% possibilities of printing a horizontal cut
                              useful to see if the skull stripping is working
                              as expected
    '''
    
    # separte the name of the file by '_'
    splitted_name = filename.strip().split('_')
    # sometimes residual MacOS files appear; ignore them
    if splitted_name[0] == '.': return
    
    # save the image ID
    image_ID = splitted_name[-1][1:-4]
    
    # sometimes empty files appear, just ignore them
    if image_ID == '': return
    # transform the ID into a int64 numpy variable for indexing
    image_ID = np.int64(image_ID)
        
    # with the ID, index the information we need
    row_index = description.index[description['Image Data ID'] == image_ID].tolist()[0]
    # obtain the corresponding row in the dataframe
    row = description.iloc[row_index]
    # get the label
    label = row['Group']
    
    # prepare the origin path
    complete_file_path = os.path.join(path, filename)
    
    if skull_stripping:
        complete_new_path = os.path.join(DRIVE_SS_PATH, 
                                         label,
                                         filename)
        skull_strip_nii(complete_file_path, complete_new_path)
    else:
        complete_new_path = os.path.join(DRIVE_SHELL_PATH, label)
        # copy the image to the drive folder
        ! cp $complete_file_path $complete_new_path
        
    # print the image 10% of the time
    if random_printing and np.random.randint(0, 101) > 90:
        sitk_image = sitk.ReadImage(complete_new_path)
        img = sitk.GetArrayFromImage(sitk_image)
        plt.figure(figsize=(10,10))
        plt.imshow(img[:, :, np.random.randint(70, 160)], 
                   cmap='gray')
        plt.show()

Now, several for loops are written for processing and uploading all of the images. Save exceptions, so the loop does not stop with an error. Later, exceptions can be checked to see which images could not be processed.

In [0]:
exceptions = []
for subdir in DATASET_FOLDERS:
    for path, dirs, files in os.walk(DATASET_PATH + subdir):
        if files:
            for file in files:
                try:
                    process_and_upload(file, path, 
                                       skull_stripping=False, 
                                       random_printing=True)
                except RuntimeError:
                    exceptions.append(os.path.join(path, file))

---

## Image registration

### Testing

In this section, the `SimpleElastix` package from the `SimpleITK` library is used to perform image registration. The installation of `SimpleElastix` is quite long, but necessary for this code to work.

First, define the names and paths of several images.

In [0]:
ROOT = '/path/to/root/directory'
CN_IMGS = ['CN/someimage.nii',
           'CN/someimage.nii',
           'CN/someimage.nii']
MCI_IMGS = ['MCI/someimage.nii',
            'MCI/someimage.nii',
            'MCI/someimage.nii']
AD_IMGS = ['AD/someimage.nii',
           'AD/someimage.nii',
           'AD/someimage.nii']

Load two of them. The objective is to register the moving image to the fixed one.

In [0]:
sitk_moving = sitk.ReadImage(ROOT + CN_IMGS[0])
sitk_fixed = sitk.ReadImage(ROOT + CN_IMGS[1])

moving_img = sitk.GetArrayFromImage(sitk_moving)
fixed_img = sitk.GetArrayFromImage(sitk_fixed)

print('Fixed', fixed_img.shape)
print('Moving', moving_img.shape)

Try `SimpleElastix`, to see how it performs.

In [0]:
elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.SetFixedImage(sitk_fixed)
elastixImageFilter.SetMovingImage(sitk_moving)

parameterMapVector = sitk.VectorOfParameterMap()
parameterMapVector.append(sitk.GetDefaultParameterMap("affine"))
# the following line is used for non-rigid registration
# it is commented because it is very slow and not very useful
#parameterMapVector.append(sitk.GetDefaultParameterMap("bspline"))
elastixImageFilter.SetParameterMap(parameterMapVector)

elastixImageFilter.Execute()
result = elastixImageFilter.GetResultImage()

In [0]:
img = sitk.GetArrayFromImage(result)
img.shape

Now, compare the result with the original image.

In [0]:
plt.figure(figsize=(10, 10))
plt.imshow(img[:, :, 100], cmap='gray')
plt.show()

In [0]:
plt.figure(figsize=(10, 10))
plt.imshow(moving_img[:, :, 136], cmap='gray')
plt.show()


Affine registration is quite faster, and does not make strange deformations to the images. It looks like it also works very well. So define the registration method, offering the option to use non-rigid registration by changing a parameter.

In [0]:
def registrate(sitk_fixed, sitk_moving, bspline=False):
    ''' Perform image registration using SimpleElastix.
        By default, uses affine transformation.
        
        Parameters:
            sitk_fixed -- Reference atlas (sitk .nii)
            sitk_moving -- Image to be registrated
                           (sitk .nii)
            bspline -- Whether or not to perform non-rigid
                       registration. Note: it usually deforms
                       the images and takes a lot of time
    '''
    
    elastixImageFilter = sitk.ElastixImageFilter()
    elastixImageFilter.SetFixedImage(sitk_fixed)
    elastixImageFilter.SetMovingImage(sitk_moving)

    parameterMapVector = sitk.VectorOfParameterMap()
    parameterMapVector.append(sitk.GetDefaultParameterMap("affine"))
    if bspline:
        parameterMapVector.append(sitk.GetDefaultParameterMap("bspline"))
    elastixImageFilter.SetParameterMap(parameterMapVector)

    elastixImageFilter.Execute()
    return elastixImageFilter.GetResultImage()

Try the method.

In [0]:
results = [fixed_img]
for img_name in imgs:
    moving = sitk.ReadImage(ROOT + img_name)
    result = registrate(sitk_fixed, moving)
    results.append(sitk.GetArrayFromImage(result))

In [0]:
for result in results:
    plt.figure(figsize=(10, 10))
    plt.imshow(result[:, :, 100], cmap='gray')
    plt.show()

Using affine registration seems to be enough, we can ignore non-rigid registration. Using registration means the images will not be cropped. The registration process should force all of them to have the same shape.


### More testing

Create the necessary constants to acces the data:

In [0]:
DATABASE = '/Volumes/0SC4R/ADNI/RAW/'
DB_SUBFOLDERS = ['1/', '2/', '3/', '4/', '5/']

Get a reference image.

In [0]:
FIXED_IMAGE = 'local/path/to/fixed.nii'
sitk_fixed = sitk.ReadImage(DATABASE + FIXED_IMAGE)

Let´s perform some tests to see how the images adapt to the reference. Paths are extracted from the previously created file with the shape of each image. Each one of the tests are performed with an image with a different shape.

In some cases, you may see a reference to a variable called `atlas`. That is created in the next section. We came back to this one to test how the images adapted to a reference atlas (MNI 305).

#### Image with shape [124, 256, 256]

In [0]:
path = 'image.nii'
sitk_moving = sitk.ReadImage(path)

result = registrate(atlas, sitk_moving)
original = sitk.GetArrayFromImage(sitk_fixed)
img = sitk.GetArrayFromImage(result)
print(img.shape)

In [0]:
plt.figure(figsize=(10, 10))
plt.imshow(img[50, :, :], cmap='gray')
plt.show()

#### Image with shape [184, 256, 256]

In [0]:
path = 'image.nii'
sitk_moving = sitk.ReadImage(path)

result = registrate(sitk_fixed, sitk_moving)
original = sitk.GetArrayFromImage(sitk_fixed)
img = sitk.GetArrayFromImage(result)
print(img.shape)

In [0]:
plt.figure(figsize=(10, 10))
plt.imshow(img[:, :, 60], cmap='gray')
plt.show()

#### Image with shape [146, 256, 256]

In [0]:
path = 'image.nii'
sitk_moving = sitk.ReadImage(path)

result = registrate(sitk_fixed, sitk_moving)
original = sitk.GetArrayFromImage(sitk_fixed)
img = sitk.GetArrayFromImage(result)
print(img.shape)

In [0]:
plt.figure(figsize=(10, 10))
plt.imshow(img[:, :, 70], cmap='gray')
plt.show()

#### Image with shape [170, 256, 256]

In [0]:
path = 'image.nii'
sitk_moving = sitk.ReadImage(path)

result = registrate(atlas, sitk_moving)
original = sitk.GetArrayFromImage(sitk_fixed)
img = sitk.GetArrayFromImage(result)
print(img.shape)

In [0]:
plt.figure(figsize=(10, 10))
plt.imshow(img[50, :, :], cmap='gray')
plt.show()

#### Image with shape [166, 256, 256]

These are the most common, so it is important to make sure the registration process works well.

In [0]:
path = 'image.nii'
sitk_moving = sitk.ReadImage(path)

result = registrate(sitk_fixed, sitk_moving)
original = sitk.GetArrayFromImage(sitk_fixed)
img = sitk.GetArrayFromImage(result)
print(img.shape)

In [0]:
plt.figure(figsize=(10, 10))
plt.imshow(img[:, :, 70], cmap='gray')
plt.show()

#### Trying with MNI 305 mean atlas

In [0]:
atlas_path = '/path/to/atlas.nii'
atlas = sitk.ReadImage(atlas_path)

In [0]:
path = 'image.nii'
sitk_moving = sitk.ReadImage(path)

result = registrate(atlas, sitk_moving)
img = sitk.GetArrayFromImage(result)
original = sitk.GetArrayFromImage(sitk_moving)
print(img.shape)

In [0]:
plt.figure(figsize=(10, 10))
plt.imshow(img[80, :, :], cmap='gray')
plt.show()

---

## OASIS Database (DEPRECATED)

This code was going to be used for building the OASIS database, but in the end this idea was discarded to focus on the ADNI database, which was much more complete and clear. 

In [0]:
OASIS_DB = '/path/to/OASIS/central.xnat.org/'
OASIS_RAW = '/destination/path/for/OASIS/MRI/'
SUBFOLDERS = ['AD', 'CN']
DESCRIPTION = 'Description.csv'

LABELS = {'Cognitively normal': 'CN', 'AD Dementia': 'AD'}

In [0]:
filenames = np.array([])
for path, _, files in os.walk(OASIS_DB):
    files_paths = [os.path.join(path, name) for name in files]
    filenames = np.concatenate((filenames, files_paths), axis=None)

In [0]:
mri_files = np.array([name for name in filenames if 'pet' not in name])
description = pd.read_csv(os.path.join(OASIS_RAW, DESCRIPTION))

In [0]:
import re

for name in mri_files:
    split_path = name.split('/')
    subject = split_path[-1][4:12]
    diagnosis = description.loc[description['Subject'] == subject, ['dx1']].iloc[0][0]
    if diagnosis in LABELS.keys():
        dest = os.path.join(OASIS_RAW, LABELS[diagnosis])
        ! cp $name $dest

### Building the OASIS database

In [0]:
OASIS_REGISTERED = '/path/to/OASIS/REGISTERED/'
SUBFOLDERS = ['AD', 'CN']
DESCRIPTION = 'Description.csv'

In [0]:
filenames_ad = np.array([])
filenames_cn = np.array([])
for path, _, files in os.walk(OASIS_RAW):
    adfiles = [os.path.join(path, name) for name in files if 'AD' in path]
    cnfiles = [os.path.join(path, name) for name in files if 'CN' in path]
    filenames_ad = np.concatenate((filenames_ad, adfiles), axis=None)
    filenames_cn = np.concatenate((filenames_cn, cnfiles), axis=None)

In [0]:
atlas = sitk.ReadImage('/path/to/atlas.nii')
atlas = resample_img(atlas)
exceptions = []

In [0]:
reg_path = os.path.join(OASIS_REGISTERED, 'AD')
for img in filenames_ad:
    sitk_img = sitk.ReadImage(img)
    sitk_img = resample_img(sitk_img)
    try:
        registrated = registrate(sitk_fixed=atlas, sitk_moving=sitk_img)
        name = img.split('/')[-1]
        sitk.WriteImage(registrated, os.path.join(reg_path, name))
    except:
        exceptions.append(img)

In [0]:
print(exceptions)

In [0]:
reg_path = os.path.join(OASIS_REGISTERED, 'CN')
for img in filenames_cn:
    sitk_img = sitk.ReadImage(img)
    sitk_img = resample_img(sitk_img)
    try:
        registrated = registrate(sitk_fixed=atlas, sitk_moving=sitk_img)
        name = img.split('/')[-1]
        sitk.WriteImage(registrated, os.path.join(reg_path, name))
    except:
        exceptions.append(img)

In [0]:
# save the exceptions
with open(os.path.join(OASIS_RAW, 'exceptions.txt'), 'w') as f:
    for item in exceptions:
        f.write("%s\n" % item)

---

## Building the unsupervised database (DEPRECATED)

This section was going to be used for building an unsupervised database with the IXI-T1 dataset. However, this ended up not being used in the final work, because autoencoders were discarded as one of the main methods.

In [0]:
atlas = sitk.ReadImage('/path/to/atlas.nii')
# resample the atlas to the desired spatial resolution
atlas = resample_img(atlas)

In [0]:
DB = '/path/to/database/'
DEST = '/path/to/destination/'
exceptions = []

for file in os.listdir(DB):
    try:
        name = os.path.join(DB, file)
        new_name = os.path.join(DEST, file)
        sitk_moving = sitk.ReadImage(name)
        sitk_moving = resample_img(sitk_moving)
        registrated = registrate(atlas, sitk_moving)
        sitk.WriteImage(registrated, new_name)
    except:
        exceptions.append(name)

In [0]:
# save the exceptions
with open(os.path.join(DB, 'exceptions.txt'), 'w') as f:
    for item in exceptions:
        f.write("%s\n" % item)

---

## Building the ADNI database

The ADNI database was the one used in the final work. It is built using the following cells, in which the images are resampled, registered to the MNI 305 atlas and saved in the corresponding folder, which would be the one of its label (AD/CN/MCI). The, the resulting folder structure is manually uploaded to Google Drive, so it can be used with Google Colaboratory.

Define the paths where the images are, and where to store them.

In [0]:
# original database
DATABASE = '/Volumes/0SC4R/TFM-Data/ADNI/MRI/RAW/'
DB_SUBFOLDERS = ['1/', '2/', '3/', '4/', '5/', '6/',
                 '7/', '8/', '9/', '10/', '11/', '12/',
                 '13/', '14/4', '15/', '16/', '17/', 
                 '18/', '19/', '20/']

# registered and organized database
REG_DB = '/Volumes/0SC4R/TFM-Data/ADNI/MRI/REGISTERED/'
REG_DB_SUBFOLDERS = ['AD/', 'MCI/', 'CN/']

Load and resample the 305 MNI atlas, as well as the description file.

In [0]:
atlas = sitk.ReadImage('/path/to/atlas.nii')
atlas = resample_img(atlas)

description = pd.read_csv('/Volumes/0SC4R/TFM-Data/ADNI/MRI/Description.csv')
description.head()

In [0]:
def register_and_save(filename, path, atlas, random_printing=False):
    ''' Process the image name and copy the image to its
        corresponding Google Drive folder.
        
        Parameters:
            filename -- Name of the image file (.nii)
            path -- The path were the image is located
            atlas -- Reference sitk image for registration
            random_priting -- 10% possibilities of printing a horizontal cut
                              useful to see if the skull stripping is working
                              as expected
    '''
    
    # separte the name of the file by '_'
    splitted_name = filename.strip().split('_')
    # sometimes residual MacOS files appear; ignore them
    if splitted_name[0] == '.': return
    
    # save the image ID
    image_ID = splitted_name[-1][1:-4]
    
    # sometimes empty files appear, just ignore them
    if image_ID == '': return
    # transform the ID into a int64 numpy variable for indexing
    image_ID = np.int64(image_ID)
        
    # with the ID, index the information we need
    row_index = description.index[description['Image Data ID'] == image_ID].tolist()[0]
    # obtain the corresponding row in the dataframe
    row = description.iloc[row_index]
    # get the label
    label = row['Group']
    
    # prepare the origin path
    complete_file_path = os.path.join(path, filename)
    # load sitk image
    sitk_moving = sitk.ReadImage(complete_file_path)
    sitk_moving = resample_img(sitk_moving)
    registrated = registrate(atlas, sitk_moving)
    
    # prepare the destination path
    complete_new_path = os.path.join(REG_DB, 
                                     label,
                                     filename)
    sitk.WriteImage(registrated, complete_new_path)
        
    if random_printing and np.random.randint(0, 101) > 90:
        sitk_image = sitk.ReadImage(complete_new_path)
        img = sitk.GetArrayFromImage(sitk_image)
        plt.figure(figsize=(10,10))
        plt.imshow(img[np.random.randint(30, 70), :, :], 
                   cmap='gray')
        plt.show()

In [0]:
for subdir in DB_SUBFOLDERS:
    for path, dirs, files in os.walk(DATABASE + subdir):
        if files:
            for file in files:
                try:
                    register_and_save(file, 
                                      path, 
                                      atlas,
                                      random_printing=False)
                except RuntimeError:
                    print('Exception with', os.path.join(path, file))

---

## Skull stripping

Section for skull stripping experimentation and execution. The FSL BET interface of `nipype` is used, so FSL BET must be installed in the local machine for this to work.

In [0]:
from nipype.interfaces import fsl
import matplotlib.pyplot as plt

The following is the main method for skull stripping. It is pretty simple, and allows for easy modification of the fractional intensity threshold.

In [0]:
def skull_strip_nii(original_img, destination_img, frac=0.3):
    ''' Practice skull stripping on the given image, and save
        the result to a new .nii image.
        Uses FSL-BET 
        (https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/BET/UserGuide#Main_bet2_options:)
        
        Parameters:
            original_img -- Original nii image
            destination_img -- The new skull-stripped image
            frac -- Fractional intensity threshold for BET
    '''
    
    btr = fsl.BET()
    btr.inputs.in_file = original_img
    btr.inputs.frac = frac
    btr.inputs.out_file = destination_img
    btr.cmdline
    res = btr.run()

#### Supervised data

Now, apply the method on the supervised data, which is the one that ended up being used in the final work. Just define the origin and destination paths.

In [0]:
REG_DB = '/Volumes/0SC4R/TFM-Data/ADNI/MRI/REGISTERED/'
SKULL_STRIPPED_DB = '/Volumes/0SC4R/TFM-Data/ADNI/MRI/SKULL_STRIPPED/'
CLASS_FOLDERS = ['AD', 'MCI', 'CN']

Now, iterate through all files from the origin folders, perform skull stripping and save. Be sure to manage the possible errors by saving all the exceptions to a list, and then export it. 

In [0]:
exceptions = []
for folder in CLASS_FOLDERS:
    origin_folder = os.path.join(REG_DB, folder)
    dest_folder = os.path.join(SKULL_STRIPPED_DB, folder)
    for path, _, files in os.walk(origin_folder):
        for file in files:
            try:
                img = os.path.join(path, file)
                dest = os.path.join(dest_folder, file)
                skull_strip_nii(img, dest, frac=0.2)
            except RuntimeError:
                exceptions.append(img)

# save the exceptions
with open(os.path.join(SKULL_STRIPPED_DB, 'exceptions.txt'), 'w') as f:
    for item in exceptions:
        f.write("%s\n" % item)

#### Unsupervised data (DEPRECATED)

The unsupervised data was not used in the end, so this code could be defined as deprecated.

In [0]:
REG_DB = '/Volumes/0SC4R/TFM-Data/IXI-T1/REGISTERED/'
SKULL_STRIPPED_DB = '/Volumes/0SC4R/TFM-Data/IXI-T1/SKULL-STRIPPED/'

files = [os.path.join(REG_DB, name) for name in os.listdir(REG_DB)]

In [0]:
exceptions = []
for filename in files:
    try:
        dest = os.path.join(SKULL_STRIPPED_DB, filename.split('/')[-1])
        skull_strip_nii(filename, dest, frac=0.5)
        
        result = sitk.ReadImage(dest)
        img = sitk.GetArrayFromImage(result)
        plt.imshow(img[np.random.randint(20, 70), :, :], cmap='gray')
        plt.show()
    except:
        exceptions.append(filename)

# save the exceptions
with open(os.path.join(SKULL_STRIPPED_DB, 'exceptions.txt'), 'w') as f:
    for item in exceptions:
        f.write("%s\n" % item)

---

## Working with PET images (DEPRECATED)

Using MRI images originally resulted in very bad results, so a differente approach was tried. The attention switched to PET images obtained from the ADNI database. The dataset consists of 1251 images, post-processed with spatial normalization, baseline alignment and Tx Origin. In the end, a few tweaks allowed to obtain much better results with MRI images, so this code is not a part of the final work.

The localization of the data is in the following constants:

In [0]:
PET_DB_PATH = '/Volumes/0SC4R/TFM-Data/ADNI/PET/'
DESCRIPTION = 'PET.csv'
DATA_FOLDER = 'RAW/'

We load the `.csv` file with all the information about the images.

In [0]:
pet_data = os.path.join(PET_DB_PATH, DATA_FOLDER)
description = pd.read_csv(os.path.join(PET_DB_PATH, DESCRIPTION))
description.set_index('Subject ID', inplace=True)
description.head()

Save the complete path of every image in a numpy array, so that information becomes easily accesible.

In [0]:
images = np.array([])
for path, _, files in os.walk(pet_data):
    images = np.concatenate((images, 
                             [os.path.join(path, name) 
                                  for name in files]), 
                            axis=None)

In [0]:
rand = np.random.choice(images)
sitk_image = sitk.ReadImage(rand)
img = sitk.GetArrayFromImage(sitk_image)

plt.figure(figsize=(5, 5))
plt.imshow(img[55, :, :])
plt.show()

In [0]:
sitk_img = resample_img(itk_image=sitk_image)
img = sitk.GetArrayFromImage(sitk_image)
img.shape

All 1251 images have the same shape of 69x95x79. They seem to have been already resampled to $2mm^3$ isotropic voxels, because using resampling does not change the resolution. They all have also the same scaling, meaning that we do not need to registrate them. The only thing we have to do is organize them into folders according to their class:

In [0]:
ORGANIZED_FOLDER = 'Organized/'
destination = os.path.join(PET_DB_PATH, ORGANIZED_FOLDER)

In [0]:
def get_label_for(filename):
    ''' Returns the label for a given image.
        In this case we do not have image IDs, for some
        reason, so we need to infer the label by using
        the subject ID.
        
        Parameters:
            filename -- Complete path and filename
        
        Returns:
            label (CN/MCI/AD)
    '''
    
    path_folders = filename.split('/')
    image_name = path_folders[-1]
    image_name = image_name.split('_')
    subject_ID = '_'.join([image_name[1], 
                           image_name[2], 
                           image_name[3]])
    label = description.loc[subject_ID, 'Research Group']
    if type(label) is str: return label
    else: return label.iloc[0]

In [0]:
for filename in images:
    label = get_label_for(filename)
    copy_to = os.path.join(destination, label)
    ! cp $filename $copy_to

Now, the information can be uploaded to Google Drive. The images are ready to be used by a CNN, although it may be necessary to create a TFRecord, because there is too much data. Images also need obvious extra processing if they are going to be fed to a 3D RGB CNN.

---